## Read data and create dataframe

In [146]:
import dbf
import csv

In [147]:
filename = "SINASC13_GEO.dbf"

In [148]:
import pysal as ps
import pandas as pd
'''
Arguments
---------
dbfile  : DBF file - Input to be imported
upper   : Condition - If true, make column heads upper case
'''
def dbf2DF(dbfile, upper=True): #Reads in DBF files and returns Pandas DF
    db = ps.open(dbfile) #Pysal to open DBF
    d = {col: db.by_col(col) for col in db.header} #Convert dbf to dictionary
    #pandasDF = pd.DataFrame(db[:]) #Convert to Pandas DF
    pandasDF = pd.DataFrame(d) #Convert to Pandas DF
    if upper == True: #Make columns uppercase if wanted 
        pandasDF.columns = map(str.upper, db.header) 
    db.close() 
    return pandasDF

In [149]:
uda = dbf2DF(filename, 'True')

In [150]:
print(uda.axes)

[RangeIndex(start=0, stop=172457, step=1), Index([u'APGAR5', u'CODESTAB', u'CODMUNNASC', u'COMUNESTAB', u'COORD_X',
       u'COORD_Y', u'DTNASC', u'ESCMAE', u'ESCMAE2010', u'ESTCIVMAE',
       u'GRAVIDEZ', u'IDADEMAE', u'PARTO', u'PESO', u'QTDPARTCES',
       u'QTDPARTNOR', u'RACACOR', u'RACACORMAE', u'SEMAGESTAC', u'SEXO',
       u'STCESPARTO', u'STTRABPART', u'TPAPRESENT', u'TPROBSON'],
      dtype='object')]


## Initial EDA 

In [155]:
len(uda)

172457

Dataset with PARTO and GRUPO

In [164]:
partoEGrupo = uda.loc[uda['PARTO'].isin(['1', '2'])][['PARTO','TPROBSON']]

Missing data on PARTO

In [304]:
uda['QTDPARTCES'].value_counts()

00    131624
01     32219
02      6695
03      1437
04       280
         121
05        49
06        11
10         8
11         4
07         4
30         2
35         1
20         1
99         1
Name: QTDPARTCES, dtype: int64

In [184]:
partosPorGrupo = partoEGrupo.groupby(['PARTO','TPROBSON']).size(
                             ).reset_index(
                             ).rename(columns={0:'PARTOS'}
                             ).sort_values(by=['TPROBSON', 'PARTOS'])

In [185]:
totalPartosPorGrupo = partosPorGrupo.groupby('TPROBSON').sum(
                        ).reset_index(
                        ).rename(columns={'PARTOS': 'TOTAL_GRUPO'})

In [186]:
totalPartosPorGrupo.head()

,TPROBSON,TOTAL_GRUPO
0,01,17566
1,02,52095
2,03,16912
3,04,22239
4,05,34531


In [187]:
partosPorGrupo = pd.merge(partosPorGrupo, totalPartosPorGrupo, on='TPROBSON')

In [188]:
partosPorGrupo['%'] = partosPorGrupo['PARTOS']/partosPorGrupo['TOTAL_GRUPO']

In [191]:
partosPorGrupo.sort_values(by=['TPROBSON', 'PARTO'])

,PARTO,TPROBSON,PARTOS,TOTAL_GRUPO,%
1,1,01,10411,17566,0.592679
0,2,01,7155,17566,0.407321
2,1,02,19162,52095,0.367828
3,2,02,32933,52095,0.632172
5,1,03,14794,16912,0.874763
4,2,03,2118,16912,0.125237
7,1,04,16056,22239,0.721975
6,2,04,6183,22239,0.278025
8,1,05,6067,34531,0.175697
9,2,05,28464,34531,0.824303
